# Maximum Likelihood Estimation

In [1]:
import pandas as pd
import numpy as np  
from scipy.optimize import minimize  

import csv
import scipy

In [2]:
df = pd.read_parquet("final_data.parquet")
df

,constructorId,constructor_name,constructorStandingsId,raceId_x,constructor_pos,constructor_wins,constructorResultsId,constructor_points,round,circuitId,...,Points_Per_Entry,Years_Active,Champion,driverRef,code,driverStandingsId,raceId_y,points,position,driver_wins
0,1,McLaren,26936.0,989.0,4.0,0.0,15643.0,12.0,1,1,...,5.78273,19,True,alonso,ALO,4,18,5.0,4,0
1,1,McLaren,26936.0,989.0,4.0,0.0,15643.0,12.0,1,1,...,5.78273,19,True,alonso,ALO,12,19,6.0,7,0
2,1,McLaren,26936.0,989.0,4.0,0.0,15643.0,12.0,1,1,...,5.78273,19,True,alonso,ALO,30,20,6.0,9,0
3,1,McLaren,26936.0,989.0,4.0,0.0,15643.0,12.0,1,1,...,5.78273,19,True,alonso,ALO,51,21,6.0,10,0
4,1,McLaren,26936.0,989.0,4.0,0.0,15643.0,12.0,1,1,...,5.78273,19,True,alonso,ALO,72,22,9.0,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355645,3,Williams,28709.0,1120.0,7.0,0.0,16887.0,0.0,22,24,...,0.00000,1,False,sargeant,SAR,73128,1140,0.0,22,0
355646,3,Williams,28709.0,1120.0,7.0,0.0,16887.0,0.0,22,24,...,0.00000,1,False,sargeant,SAR,73174,1141,0.0,22,0
355647,3,Williams,28709.0,1120.0,7.0,0.0,16887.0,0.0,22,24,...,0.00000,1,False,sargeant,SAR,73220,1142,0.0,22,0
355648,3,Williams,28709.0,1120.0,7.0,0.0,16887.0,0.0,22,24,...,0.00000,1,False,sargeant,SAR,73243,1143,0.0,23,0


In [3]:
missing_columns = df.columns[df.isna().any()]
print("Columns with missing or NaN values:")
print(missing_columns)

Columns with missing or NaN values:
Index(['AirTemp', 'Humidity', 'Pressure', 'Rainfall', 'TrackTemp',
       'WindDirection', 'WindSpeed'],
      dtype='object')


## Predicting the probability that drivers will place in the top 3

Perform binary classification for whether a driver/constructor duo has placed in the top 3

In [5]:
df['results_pos'] = pd.to_numeric(df['results_pos'], errors='coerce')
df = df.fillna(22)
df['results_pos'] = df['results_pos'].astype(int)
df['top_3'] = (df['results_pos'] < 4)